# Configuration file parser

PhysiCOOL introduces two data structures to interact with the `PhysiCell_settings.xml` file:

- The `CellParameters` class, which stores all the variables for a given cell definition;
- The `ConfigFileParser` which lets users read and write to the config file.

Although these classes were developed to be part of the PhysiCOOL black-box model, they can be used on their own to programatically change the values in the PhysiCell configuration file.

## Reading cell data

Reading data from the config file can be done by creating a `ConfigFileParser` instance, which takes as input the path to the file. This class has a function called `read_cell_data`, whcih will read the data for a cell definition (passed as input) and store it in a Python data structure, the `CellParameters` class.

Cell parameters are divided into groups as in the settings file (e.g., motility, mechanics, secretion,...), which can be accessed independently. The following commands read the data from a config file and print out the motility parameters.

In [1]:
from physicool.config import ConfigFileParser

file_path = "../tests/PhysiCell/config/PhysiCell_settings.xml"

# Parse the data from the config file
xml_data = ConfigFileParser(file_path)

# Read the cell parameters for the "default" cell definition
cell_data = xml_data.read_cell_data("default")

print("Print full cell data:")
print(cell_data)
print("\nAccess and print just the motility data:")
print(cell_data.motility)

Print full cell data:
CellParameters(name='default', volume=Volume(total_volume=2494.0, fluid_fraction=0.75, nuclear=540.0, fluid_change_rate=0.05, cyto_bio_rate=0.0045, nuclear_bio_rate=0.0055, calcified_fraction=0.0, calcification_rate=0.0, rupture_volume=2.0), mechanics=Mechanics(adhesion_strength=0.4, repulsion_strength=10.0, adhesion_distance=1.25), motility=Motility(speed=20.0, persistence=25.0, bias=0.5, motility_enabled=False, use_2d=True, chemo_enabled=False, chemo_substrate='substrate', chemo_direction=1.0), secretion=[Secretion(name=substrate, secretion_rate=0.0,secretion_target=1.0, uptake_rate=0.0,net_export_rate=0.0)])

Access and print just the motility data:
Motility(speed=20.0, persistence=25.0, bias=0.5, motility_enabled=False, use_2d=True, chemo_enabled=False, chemo_substrate='substrate', chemo_direction=1.0)


## Writing new data values

Each parameter can also be accessed independently and their values can be changed. To write these chanegs to the XML file, the `ConfigFileParser` class includes a function called `update_params`, which takes as input the updated cell data structure and the name of the cell definition.

For instance, the cell speed and the 2D status can be changed with the following commands:

In [2]:
# Update and write the new parameters to the config file
cell_data.motility.speed = 20.0
cell_data.motility.use_2d = True

xml_data.update_params(name="default", new_parameters=cell_data)